In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/carlo/workspace/networkqit/')
import numpy as np
import pandas as pd
import scipy
import networkqit as nq
import operator
import matplotlib.pyplot as plt
from itertools import product
from networkqit.utils.visualization import step_callback, plot_spectral_entropy
from networkqit import graph_laplacian
import seaborn as sns
import multiprocessing
import warnings
from drawnow import drawnow, figure
# Set seaborn as matplotlib backend style
sns.set()
from os.path import expanduser
from bct import threshold_proportional, threshold_absolute
home = expanduser("~")

# Define the methods to create weighted correlation networks from factor model 

In [ ]:
def factor_model(ci,T,eta,mu):
    N = len(ci) # number of nodes, length of membership vector
    # Initialize the observations vector a TxN matrix of NaNs
    Y = np.ones([T,N])*np.nan
    
    # Fill the identical observations in the maximally correlated subsets
    for c in np.unique(ci):
        i = np.where(ci==c)[0]
        Y[:,i] = np.kron(np.ones((1,(ci==c).sum())),np.random.randn(T,1))

    # Add local noise beta on each time-series
    Y += eta*np.random.randn(T,N)
        
    # Add global signal mu that correlates globally each time series
    Y += mu*np.kron(np.ones((1,N)),np.random.randn(T,1))

    from scipy.stats import zscore
    Y = zscore(Y)
    C = np.corrcoef(Y.T)
    return C

In [ ]:
def brain_network(ci, T=150, local_noise=2, global_noise=4, gamma=10, density=0.1):
    C = np.triu(factor_model(ci,T,local_noise,global_noise),1)
    C += C.T
    C = C**gamma # soft thresholding
    C/=C.max()
    C = np.arctanh(C)
    C[np.isinf(C)]=0
    C = threshold_proportional(C,density) # keep a specific percentage of links
    return C

In [ ]:
def reindex_membership(membership):
    """
    This function has the membership as input and output the membership
    where the communities number are ordered by the number of nodes in that community
    """
    ds = {}
    for u, v in enumerate(membership):
        if v not in ds.keys():
            ds[v] = []
        ds[v].append(u)

    S = dict(
        zip(range(0, len(ds)), sorted(ds.values(), key=len, reverse=True)))

    M = {}

    for u, vl in S.items():
        for v in vl:
            M[v] = u
    return list(M.values())

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=1,figsize=(20,8))
A = brain_network(ci,T=100,local_noise=2.5, global_noise=4, gamma=1,density=1)
plt.suptitle('Networks from factor model local noise $\eta=2.2$ global noise $\mu=4$')
h=ax[0].imshow(A,cmap='viridis')
ax[0].grid(False)
ax[0].set_title('Adjacency |E|=' + str(len(A.nonzero()[0])/2))
plt.colorbar(h,ax=ax[0])
ax[1].hist(A.flatten()[np.flatnonzero(A)],500,alpha=0.8)
ax[1].grid(True)
ax[1].set_title('Edge weights histogram')


In [ ]:
from scipy.io import loadmat
W=loadmat('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_matrix_unthr.mat')['RS_unthr']

In [ ]:
A = brain_network(ci=sorted(np.random.randint(0,6,638)),T=150,local_noise=1.5,global_noise=4,gamma=5,density=0.6)
plt.hist(A.sum(axis=0),80,alpha=0.5,color='r')
plt.hist(W.sum(axis=0),80,alpha=0.5,color='b')
plt.legend(['Synthetic','Crossley'])
plt.show()

In [ ]:
import bct
import sys
ci = bct.community_louvain(bct.threshold_absolute(W,0.6))
ci = reindex_membership(ci[0])
[bounds,ixes] = bct.grid_communities(np.array(ci))

In [ ]:
T,local_noise,global_noise,factor=200,2.5,1.5,0.8

A = brain_network(ci=ci, T=200, local_noise=local_noise, global_noise=global_noise, gamma=1, density=1)*factor

fig, ax = plt.subplots(ncols=3,nrows=1,figsize=(24,6))
#plt.suptitle('Networks from factor model local noise $\eta=2.2$ global noise $\mu=4$')
##### plot 0
h0 = ax[0].imshow(W[np.ix_(ixes,ixes)],cmap='viridis')
ax[0].grid(False)
ax[0].set_title('Crossley')
#### plot 1
h1 = ax[1].imshow(A[np.ix_(ixes,ixes)],cmap='viridis')
ax[1].grid(False)
ax[1].set_title('Synthetic')
##### plot 2
ax[2].hist(W.ravel(),200,alpha=0.5,color='b')
ax[2].grid(True)
ax[2].set_title('Edge weights histogram Synthetic')
ax[2].hist(A.flatten(),200,alpha=0.5,color='r')
plt.colorbar(h0,ax=ax[0])
plt.colorbar(h1,ax=ax[1])
plt.legend(['Crossley','Synthetic'])
plt.show()

In [ ]:
from scipy.linalg import eigvalsh
entropy(eigvalsh(W),eigvalsh(brain_network(ci=ci, T=T, local_noise=local_noise, global_noise=global_noise, gamma=1, density=1)*factor))

In [ ]:
from scipy.stats import entropy
def klhistdiv(ci,T, local_noise, global_noise, gamma, density, factor, empirical_hist):
    A = brain_network(ci, T, local_noise, global_noise, gamma, density)*factor
    w = A.flatten()[np.flatnonzero(W.flatten())]
    weights_hist, weights_bins = np.histogram(w,500,density=False)
    plt.plot(empirical_hist)
    plt.plot(weights_hist)
    dkl = entropy(empirical_hist/empirical_hist.sum(), weights_hist/weights_hist.sum())
    print('\r',T, local_noise, global_noise, gamma, density, factor,'-->',dkl,end='')
    return dkl

weights = W.flatten()[np.flatnonzero(W.flatten())]
weights_hist, weights_bins = np.histogram(weights, 500, density=False)

klhistdiv(ci,T,local_noise=local_noise,global_noise=global_noise,gamma=1,factor=factor,density=1,empirical_hist=weights_hist)
 #brain_network(ci=ci, T=100, local_noise=1.9, global_noise=1, gamma=1, density=1)*0.8
#ranges = (slice(1,4,0.5), slice(1,6,0.5))
#T = 150
#res = scipy.optimize.brute(lambda x : klhistdiv(ci, T, x[0], x[1], 1, 1, 1, weights_hist), ranges=ranges, full_output=False, finish=None)


In [ ]:
def percolation_analysis(W,thresholds):
    giant = []
    for t in thresholds:
        Wt = threshold_absolute(W,t)
        memb, sizes = bct.get_components(Wt)
        giant.append(sizes.max())
    return giant

In [ ]:
nt=100
giantW = percolation_analysis(W,np.linspace(0.0,W.max(),nt))
giantA = percolation_analysis(A,np.linspace(0.0,A.max(),nt))
plt.figure(figsize=(24,8))
plt.plot(np.linspace(0.0,W.max(),nt),giantW)
plt.plot(np.linspace(0.0,W.max(),nt),giantA)
plt.xlabel('Threshold')
plt.ylabel('Giant component')
plt.title('Percolation analysis')

# Study the spectral entropy as function of internal symmetry

In [ ]:
import networkx as nx
def asymmetric_clique(N1,N2):
    G1 = nx.complete_graph(N1)
    G2 = nx.complete_graph(N2)
    G = nx.disjoint_union(G1,G2)
    if not nx.is_empty(G1):
        n1last = (list(G1.nodes())[-1])
        n2last = (list(G2.nodes())[-1])
        G.add_edge(n1last,n1last+1)
    return nx.to_numpy_array(G)

In [ ]:
N=60
beta_range = np.logspace(-3,3,50)
vals = range(0,N+2,5)
cmap = sns.color_palette('viridis',len(vals))

In [ ]:
#plt.semilogx(1/beta_range, [nq.compute_vonneumann_entropy(L=nq.graph_laplacian(asymmetric_clique(50,50)),beta=beta) for beta in beta_range])
plt.figure(figsize=(20,8))
for i,j in enumerate(vals):
    plt.semilogx(1/beta_range, [nq.compute_vonneumann_entropy(L=nq.graph_laplacian(asymmetric_clique(N-j,N+j)),beta=beta) for beta in beta_range],color=cmap[i])
plt.legend(vals)

# Study the derivatives of spectral entropy

In [ ]:
plt.figure(figsize=(20,8))
for i,j in enumerate(vals):
    plt.semilogx(1/beta_range, [nq.compute_vonneumann_entropy_beta_deriv(L=nq.graph_laplacian(asymmetric_clique(N-j,N+j)),beta=beta) for beta in beta_range],color=cmap[i])
plt.legend(vals)

### Study the spectral entropy as function of the threshold $\tau$

In [ ]:
beta_range=np.logspace(-3, 1.5, 100)
cmap = sns.color_palette('viridis',5)
all_thresh = np.linspace(0,0.25,5)
plt.figure(figsize=(20,8))
for i,t in enumerate(all_thresh):
    plt.semilogx(1/beta_range,np.clip([nq.SpectralDivergence(Lobs=graph_laplacian(threshold_absolute(A,t)),Lmodel=graph_laplacian(threshold_absolute(W,t)),beta=b).rel_entropy 
                                   for b in beta_range ],0,20),color=cmap[i])
plt.xlabel('$1/\\beta$')
plt.legend(all_thresh)
plt.show()

### Plot until one connected component

In [ ]:
def percolation_threshold(A):
    w = np.unique((A*(A>0).astype(float)).flatten())
    iw = range(0,len(w))
    i_perc = np.where(np.array([len(get_components(threshold_absolute(A,t))[1]) for t in w]) > 1)
    return i_perc[0][0], w[i_perc][0]

In [ ]:
i_perc,w_perc = percolation_threshold(A)
print(i_perc,w_perc)